In [76]:
import numpy as np 
from collections import deque
import numba 

In [77]:
def test_deque(deq, randy, n=10000):
    idx = 0
    for i in range(n):
        if randy[i] < 0.5:
            deq.append(i)
        else:
            deq.popleft()
            idx += 1

In [116]:
def test_list(_list, randy, n=10000):
    idx = 0    
    for i in range(n):
        if randy[i] < 0.5:
            _list.append(i)
        else:
            _list[i], _list[-1] = _list[-1], _list[i]
            del _list[-1]

In [112]:
_list = [i for i in range(110000)]
deq = deque(_list)
arr = np.array(_list)
randy = np.random.random(10000)

In [115]:
%timeit test_deque(deq.copy(), randy)
%timeit test_list(_list.copy(), randy)
# %timeit test_np(arr.copy(), randy)

4.14 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
153 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [117]:
%timeit test_list(_list.copy(), randy)

4.09 ms ± 75.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
